In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelBinarizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import plot_confusion_matrix
import pandas as pd
import math
import random
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing
import numpy as np
from collections import Counter

In [3]:
data = pd.read_excel('Sentiment_Labelling.xlsx')
# check if the thext of Wav2Vec not empty is
data["Text"] = data.apply(lambda x: x["Wav2Vec"] if x["Wav2Vec"] != np.nan else x["Youtube_Text_Corrected"], axis=1)
data = data[["Text","Sentiment", "Pitch_05_Quantile", "Pitch_95_Quantile", "Pitch_Range", "pitch stdev", "Pitch_Mean", "Pitch_Median", "Jitter", "Shimmer", "Hammarberg_Index"]]
data = data.dropna(subset=["Text", "Sentiment", "Pitch_05_Quantile", "Pitch_95_Quantile", "Pitch_Range", "pitch stdev", "Pitch_Mean", "Pitch_Median", "Jitter", "Shimmer", "Hammarberg_Index"])[["Text", "Sentiment", "Pitch_05_Quantile", "Pitch_95_Quantile", "Pitch_Range", "pitch stdev", "Pitch_Mean", "Pitch_Median", "Jitter", "Shimmer", "Hammarberg_Index"]]
def sampling_k_elements(group, k=160):
    if len(group) < k:
        return group
    return group.sample(k)
sentiment = list(data['Sentiment'].values)
# apply rhe sampling_k_elements_function of the Data to balance all sentiment_label
balanced = data.groupby('Sentiment').apply(sampling_k_elements).reset_index(drop=True)
balanced = balanced[["Text", "Sentiment", "Pitch_05_Quantile", "Pitch_95_Quantile", "Pitch_Range", "pitch stdev", "Pitch_Mean", "Pitch_Median", "Jitter", "Shimmer", "Hammarberg_Index"]]
corpus_texts = balanced["Text"]
sentiment = balanced["Sentiment"]
# create array of feutres and convert it to numpy array
audio_array = balanced[["Pitch_05_Quantile", "Pitch_95_Quantile", "Pitch_Range", "Pitch_Mean", "Pitch_Median", "Jitter", "Shimmer", "Hammarberg_Index"]].to_numpy()
audio_array = np.asarray(audio_array)


In [4]:

#  create a Tf Idf Vectorizer
polarity_tfidf_vectorizer = TfidfVectorizer()
# Vectroize the texts in the data
polarity_bow_matrix = polarity_tfidf_vectorizer.fit_transform(corpus_texts)
# convert the polarity_bow_matrix to numpy array
polarity = np.asarray(polarity_bow_matrix.todense())
print(polarity.shape)
# concatenate the array of feutres with the converted texts to create the input for our model
final_input = np.concatenate((polarity, audio_array), axis=1)

(451, 2487)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(final_input, sentiment, test_size=0.2, shuffle= True, stratify=sentiment, random_state=2)

In [6]:
mLPClassifier = MLPClassifier()
mLPClassifier.fit(X_train, y_train)
# use pickle to load the model 
pickle_load = pickle.dumps(mLPClassifier)
audio_featres_model = pickle.loads(pickle_load)
predicted = mLPClassifier.predict(X_test)
"evaluation Acc.:{:.3f}".format(np.mean(predicted == y_test))

C:\Users\Bashir\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


'evaluation Acc.:0.538'

In [7]:
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

     bearish       0.48      0.48      0.48        27
     bullish       0.53      0.59      0.56        32
     neutral       0.61      0.53      0.57        32

    accuracy                           0.54        91
   macro avg       0.54      0.54      0.54        91
weighted avg       0.54      0.54      0.54        91

